In [ ]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
統合NMC-Al界面構造生成システムで生成された構造に対応

主な修正点:
1. 入力ファイル形式をCIFに変更
2. NMC組成バリアント名に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
"""

from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# =============================================

# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
INPUT_DIR = os.path.join(BASE_DIR, "structure", "output", "final_interfaces")

# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "md_analysis")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# =============================================
# 統合システム対応のファイルパス関数
# =============================================

def get_interface_files():
    """統合システムで生成された界面構造ファイルを取得"""
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.cif')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出
            # 例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
            if "Interface_" in cif_file and "_on_" in cif_file:
                interface_files.append(cif_file)
    # interface_files = []
    # if os.path.exists(INPUT_DIR):
    #     cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.xyzf')]
    #     for cif_file in cif_files:
    #         # ファイル名から材料情報を抽出
    #         # 例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
    #         if "Interface_" in cif_file and "_on_" in cif_file:
    #             interface_files.append(cif_file)
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """界面構造ファイル名から材料情報を解析"""
    # ファイル名例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
    parts = filename.replace('.cif', '').split('_')
    
    # デフォルト値
    al_component = "Al"
    nmc_component = "NMC111"
    nmc_variant = "pristine"
    miller_al = "111"
    miller_nmc = "001"
    
    try:
        # "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed" を解析
        if len(parts) >= 6:
            # Al部分の抽出
            al_idx = parts.index("Al")
            if al_idx + 1 < len(parts):
                miller_al = parts[al_idx + 1]
            
            # "on"以降のNMC部分を解析
            on_idx = parts.index("on")
            if on_idx + 1 < len(parts):
                nmc_part = parts[on_idx + 1:]
                
                # NMC基本名
                if "NMC111" in nmc_part[0]:
                    nmc_component = nmc_part[0]
                    
                    # バリアント名の構築
                    if len(nmc_part) > 1:
                        variant_parts = []
                        for i in range(1, len(nmc_part)):
                            if nmc_part[i] not in ["trimmed"] and not nmc_part[i].isdigit():
                                variant_parts.append(nmc_part[i])
                        
                        if variant_parts:
                            nmc_variant = "_".join(variant_parts)
                        
                        # Miller指数の抽出（最後の数値部分）
                        for part in reversed(nmc_part):
                            if part.isdigit() and len(part) == 3:
                                miller_nmc = part
                                break
    
    except (ValueError, IndexError) as e:
        print(f"ファイル名解析警告 ({filename}): {e}")
    
    return {
        'al_component': al_component,
        'nmc_component': nmc_component,
        'nmc_variant': nmc_variant,
        'miller_al': miller_al,
        'miller_nmc': miller_nmc,
        'base_name': f"{al_component}_{nmc_component}_{nmc_variant}"
    }

def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

def get_compression_file_path(base_name, pressure, comp_temp):
    """圧着シミュレーションファイルのパスを取得"""
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,           # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,             # 界面構造用vacuum（Å）
    "temperature": 300.0,          # 引張温度固定（K）
    "max_strain": 3.0,             # 界面用最大ひずみ
    "timestep": 1.0,               # fs
    "elastic_region": (0.005, 0.03), # 界面用弾性領域
    "quality_threshold": 0.85,     # 界面用R²値閾値
    "max_steps": 10000,            # 界面用最大ステップ数
    "output_freq": 50,             # 出力頻度
    "free_region_thickness": 36.0  # 界面構造用固定層厚さ（Å）
}

# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,         # 界面分離判定の隙間（Å）
    "max_strain_limit": 3.0,       # 最大ひずみ制限
    "min_stress_threshold": -0.5,  # 最小応力閾値（GPa）
    "separation_wait_steps": 300,  # 分離後の待機ステップ
    "stress_drop_ratio": 0.7       # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
compression_temperatures = [300]      # 圧着シミュレーション温度（K）
compression_pressures = [0.001, 0.01] # 圧着圧力（GPa）- サンプリング
high_temperatures = [500, 800]        # 高温シミュレーション温度（K）- サンプリング
tensile_temperature = 300.0           # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とNMC原子（Li, Ni, Mn, Co, O）を分離
    al_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Al']
    nmc_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['Li', 'Ni', 'Mn', 'Co', 'O']]
    
    if not al_indices or not nmc_indices:
        # 界面でない場合のフォールバック
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)),
            'nmc_region': (np.mean(z_coords), np.max(z_coords)),
            'is_interface': False
        }
    
    al_z_coords = z_coords[al_indices]
    nmc_z_coords = z_coords[nmc_indices]
    
    # Al領域とNMC領域の境界を特定
    al_center = np.mean(al_z_coords)
    nmc_center = np.mean(nmc_z_coords)
    
    if al_center < nmc_center:
        # Alが下、NMCが上
        interface_z = (np.max(al_z_coords) + np.min(nmc_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    else:
        # NMCが下、Alが上
        interface_z = (np.max(nmc_z_coords) + np.min(al_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,
        'nmc_region': nmc_region,
        'al_center': al_center,
        'nmc_center': nmc_center,
        'is_interface': True
    }

def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定
    
    Parameters:
    -----------
    atoms : ase.Atoms
        対象の原子構造
    target_free_thickness : float
        自由領域の目標厚さ（Å）
        
    Returns:
    --------
    atoms : ase.Atoms
        固定制約が設定された原子構造
    """
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得
    interface_info = identify_interface_region(atoms)
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  Al領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å")
        print(f"  NMC領域: {interface_info['nmc_region'][0]:.2f} - {interface_info['nmc_region'][1]:.2f}Å")
        
        # 界面中心を基準に自由領域を設定
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
        
        # 自由領域の範囲
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)  # 最低2Å下は固定
        free_z_max = min(free_z_max, z_max - 2.0)  # 最低2Å上は固定
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        # 固定原子の選択（自由領域外を固定）
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合：中心を基準に自由領域を設定
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        # 固定原子の選択
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
    # 固定制約の設定
    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        return atoms
    
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    # 統計情報
    free_atoms = len(atoms) - len(indices)
    free_ratio = free_atoms / len(atoms)
    fixed_ratio = len(indices) / len(atoms)
    
    print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
    print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
    # 構造情報を属性として保存
    atoms.info['free_region_thickness'] = actual_free_thickness
    atoms.info['free_region_bounds'] = (free_z_min, free_z_max)
    atoms.info['fixed_atoms_count'] = len(indices)
    atoms.info['free_atoms_count'] = free_atoms
    
    return atoms

# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_nmc_separation = False
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出"""
        
        # Al-NMC界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms)
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region']
                nmc_region = interface_info['nmc_region']
                
                # Al-NMC間の隙間を計算
                if interface_info['al_center'] < interface_info['nmc_center']:
                    gap = nmc_region[0] - al_region[1]
                else:
                    gap = al_region[0] - nmc_region[1]
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_nmc_separation:
                    self.al_nmc_separation = True
                    self.debonding_step = system.current_total_step
                    print(f"       🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        # [界面特有の解析を追加]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = -stress_tensor[2, 2] / units.GPa
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算（界面用に調整されたパラメータ）
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15:  # 界面用に閾値調整
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"       ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_nmc_separation:
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_nmc_separation,
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理"""
    
    print("=" * 80)
    print("NMC界面構造MD解析システム")
    print("統合NMC-Al界面構造生成システムとの連携版")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        print("統合NMC-Al界面構造生成システムを先に実行してください")
        return
    
    print(f"解析対象の界面構造: {len(interface_files)}個")
    
    # 処理対象のサンプリング（全て処理する場合は以下をコメントアウト）
    # interface_files = interface_files[:5]  # 最初の5個のみ処理
    
    processed_count = 0
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"材料: {material_info}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # ファイルパス設定
                            compression_file = get_compression_file_path(base_name, pressure, comp_temp)
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)

                            # ステップ1: 圧着
                            if not check_md_completed(compression_file, 8000, 100):
                                print("圧着シミュレーション実行中...")
                                ase_atoms = read(input_file)
                                ase_atoms = set_interface_fixed_atoms_uniform_free(ase_atoms, UNIFIED_CONDITIONS["free_region_thickness"])
                                run_interface_compression_md(ase_atoms, pressure, comp_temp, compression_file)
                            else:
                                print("圧着シミュレーションは既に完了しています。")

                            # ステップ2: 平衡化 & 高温処理
                            if not check_md_completed(high_temp_file, 10000, 100):
                                print(f"平衡化と高温シミュレーション({high_temp}K)を実行中...")
                                # 平衡化 (NVT, 3ps @ 300K)
                                atoms_after_comp = read(compression_file, index='-1')
                                atoms_after_comp = set_interface_fixed_atoms_uniform_free(atoms_after_comp, UNIFIED_CONDITIONS["free_region_thickness"])
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_comp)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(300.0)
                                integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                                
                                # 高温処理 (NVT, 10ps @ high_temp)
                                atoms_after_equil = read(equil_file, index='-1')
                                atoms_after_equil = set_interface_fixed_atoms_uniform_free(atoms_after_equil, UNIFIED_CONDITIONS["free_region_thickness"])
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_equil)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(float(high_temp))
                                integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                            else:
                                print("高温シミュレーションは既に完了しています。")

                            # ステップ3: 引張試験
                            if not os.path.exists(summary_file):
                                print("引張シミュレーション実行中...")
                                final_results = run_interface_tensile_md(high_temp_file, tensile_file, analysis_file)
                                save_interface_results(final_results, summary_file, base_name, material_info, pressure, comp_temp, high_temp)
                                
                                print(f"\n✅ 引張計算完了: {base_name}")
                                print(f"📊 結果サマリー:")
                                print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                            else:
                                print("引張シミュレーションと解析は既に完了しています。")

                        except Exception as e:
                            print(f"An error occurred processing {base_name} ({condition_name}): {e}")
                            import traceback
                            traceback.print_exc()
                            continue

            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def rotate_atoms_to_lower(atoms):
    """原子系のZ座標を確認し、全原子を下端（Z=0付近）に移動させる"""
    ase_atoms = atoms.ase_atoms if hasattr(atoms, 'ase_atoms') else atoms
    
    positions = ase_atoms.get_positions()
    if len(positions) == 0:
        return ase_atoms
    z_min = np.min(positions[:, 2])
    
    shift_z = -z_min
    translation = np.array([0, 0, shift_z])
    ase_atoms.translate(translation)
    
    print(f"原子をZ方向に {shift_z:.2f}Å 移動しました")
    
    return ase_atoms


def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
    """界面構造をMD計算用に準備（自由領域厚み統一版）"""
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # CIFファイルの読み込み
    ase_atoms = read(input_file, index='-1') # Read the last frame if it's a trajectory
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    
    # 構造解析
    positions = ase_atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max - z_min
    
    print(f"界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f}Å")
    print(f"  原子数: {len(ase_atoms)}")
    
    # 界面情報取得
    interface_info = identify_interface_region(ase_atoms)
    
    new_cell_z = material_thickness
    if vacuum_top > 0:
        print(f"設定: vacuum有り ({vacuum_top}Å)")
        add_vacuum(ase_atoms, vacuum=vacuum_top, axis=2)
        new_cell_z += vacuum_top
        initial_reference = material_thickness
    else:
        print(f"設定: vacuum無し")
        initial_reference = ase_atoms.get_cell()[2, 2]

    # 自由領域厚み統一の固定層設定
    ase_atoms = set_interface_fixed_atoms_uniform_free(ase_atoms, target_free_thickness)
    
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_top,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
        "interface_info": interface_info,
        "strain_method": "material_length" if vacuum_top > 0 else "cell_length",
        "free_region_thickness": ase_atoms.info.get('free_region_thickness', target_free_thickness),
        "free_region_bounds": ase_atoms.info.get('free_region_bounds', (0, target_free_thickness)),
        "fixed_atoms_count": ase_atoms.info.get('fixed_atoms_count', 0),
        "free_atoms_count": ase_atoms.info.get('free_atoms_count', 0)
    }
    
    print(f"準備完了:")
    print(f"  セルZ長: {ase_atoms.get_cell()[2,2]:.2f}Å")
    print(f"  材料占有率: {structure_info['material_ratio']*100:.1f}%")
    print(f"  界面検出: {interface_info['is_interface']}")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    
    return ase_atoms, initial_reference, structure_info


def run_interface_compression_md(ase_atoms, pressure, comp_temp, output_file):
    """界面構造用圧着MD"""
    
    ase_atoms.set_pbc(True)
    ase_atoms.wrap()
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    
    press_ase = pressure * units.GPa
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    system = ASEMDSystem(atoms)
    system.init_temperature(comp_temp)
    
    # 初期NVT平衡化
    nvt_integrator = LangevinIntegrator(timestep=1.0, temperature=comp_temp)
    md = MDFeature(nvt_integrator, n_run=1000, estimator_fn=estimator_fn)
    md(system, extensions=[(PrintCellShape(), 200)])
    
    # NPT圧着
    integrator = NPTIntegrator(
        timestep=1.0,
        temperature=comp_temp,
        pressure=press_ase,
        ttime=20*units.fs,
        pfactor=2e6*units.GPa*(units.fs**2),
        mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
    )
    
    md = MDFeature(
        integrator, 
        n_run=8000, 
        traj_file_name=output_file,
        traj_freq=100, 
        estimator_fn=estimator_fn
    )
    
    md(system, extensions=[(PrintCellShape(), 500)])
    return output_file


def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD（自由領域厚み統一版）"""
    
    # 構造準備
    ase_atoms, initial_reference, structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["free_region_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if initial_reference == 0:
        initial_reference = 50 # Fallback value to avoid division by zero
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    
    print(f"自由領域統一引張設定:")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    print(f"  自由原子数: {structure_info['free_atoms_count']}")
    print(f"  固定原子数: {structure_info['fixed_atoms_count']}")
    print(f"  引張ひずみ速度: {strain_rate} 1/ps")

    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"NMC-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  Al成分: {material_info['al_component']}\n")
        f.write(f"  NMC成分: {material_info['nmc_component']}\n")
        f.write(f"  NMC組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  Al Miller指数: {material_info['miller_al']}\n")
        f.write(f"  NMC Miller指数: {material_info['miller_nmc']}\n\n")
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        free_count = structure_info.get('free_atoms_count', 0)
        fixed_count = structure_info.get('fixed_atoms_count', 1) # Avoid division by zero
        total_atoms = free_count + fixed_count
        free_ratio = free_count / total_atoms if total_atoms > 0 else 0

        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {free_count}\n")
        f.write(f"  固定原子数: {fixed_count}\n")
        f.write(f"  自由原子比率: {free_ratio * 100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
            
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        
        f.write(f"\n自由領域統一システムの利点:\n")
        f.write(f"  統一自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']}Å\n")
        f.write(f"  公平な機械特性比較が可能\n")
        f.write(f"  界面中心を基準とした適応的固定層設定\n")
        f.write(f"  材料厚さの違いを考慮した統一評価\n")
        f.write(f"  計算効率と比較性の両立\n")

# 使用例とテスト関数
def test_uniform_free_region():
    """自由領域統一システムのテスト関数"""
    print("自由領域統一システムのテスト開始")
    print("-" * 50)
    
    # テスト用の仮想界面構造を作成
    from ase import Atoms
    
    # Al部分（下部）
    al_positions = []
    for z in np.arange(0, 10, 2):
        for x in np.arange(0, 6, 2):
            for y in np.arange(0, 6, 2):
                al_positions.append([x, y, z])
    
    # NMC部分（上部）
    nmc_positions = []
    elements = ['Li', 'Ni', 'Mn', 'Co', 'O']
    for z in np.arange(12, 25, 2):
        for x in np.arange(0, 6, 2):
            for y in np.arange(0, 6, 2):
                nmc_positions.append([x, y, z])
    
    # 原子構造作成
    positions = al_positions + nmc_positions
    symbols = ['Al'] * len(al_positions) + elements * (len(nmc_positions) // len(elements) + 1)
    symbols = symbols[:len(positions)]
    
    test_atoms = Atoms(symbols=symbols, positions=positions)
    test_atoms.set_cell([8, 8, 30])
    test_atoms.set_pbc(True)
    
    print(f"テスト構造: {len(test_atoms)} atoms, 厚さ {np.max(test_atoms.positions[:, 2]) - np.min(test_atoms.positions[:, 2]):.1f}Å")
    
    # 自由領域統一固定の適用
    target_free = 20.0
    result_atoms = set_interface_fixed_atoms_uniform_free(test_atoms, target_free)
    
    # 結果確認
    print(f"\n結果:")
    print(f"  目標自由領域厚さ: {target_free}Å")
    print(f"  実際の自由領域厚さ: {result_atoms.info.get('free_region_thickness', 'N/A')}Å")
    print(f"  自由/固定比率: {result_atoms.info.get('free_atoms_count', 0)}/{result_atoms.info.get('fixed_atoms_count', 0)}")
    
    return result_atoms


def create_comparison_report(md_results_dir):
    """自由領域統一による結果比較レポートを作成"""
    
    if not os.path.exists(md_results_dir):
        print(f"MD結果ディレクトリが見つかりません: {md_results_dir}")
        return
    
    summary_files = list(Path(md_results_dir).glob("tensile/*_summary.txt"))
    if not summary_files:
        print("解析結果ファイルが見つかりません")
        return
    
    print(f"自由領域統一比較レポート作成中...")
    
    results_data = []
    for summary_file in summary_files:
        try:
            with open(summary_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # 基本情報の抽出
            result = {'filename': summary_file.stem}
            
            # 自由領域情報の抽出
            if '実際の自由領域厚さ:' in content:
                line = [l for l in content.split('\n') if '実際の自由領域厚さ:' in l][0]
                result['actual_free_thickness'] = float(line.split(':')[1].split('Å')[0].strip())
            
            if '自由原子数:' in content:
                line = [l for l in content.split('\n') if '自由原子数:' in l][0]
                result['free_atoms'] = int(line.split(':')[1].strip())
            
            if '固定原子数:' in content:
                line = [l for l in content.split('\n') if '固定原子数:' in l][0]
                result['fixed_atoms'] = int(line.split(':')[1].strip())
            
            # 機械特性の抽出
            if 'ヤング率:' in content:
                line = [l for l in content.split('\n') if 'ヤング率:' in l][0]
                result['young_modulus'] = float(line.split(':')[1].split('GPa')[0].strip())
            
            results_data.append(result)
            
        except Exception as e:
            print(f"ファイル処理エラー {summary_file.name}: {e}")
    
    # 統計レポート
    if results_data:
        df = pd.DataFrame(results_data)
        
        report_path = Path(md_results_dir).parent / "free_region_comparison_report.txt"
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("自由領域統一システム比較レポート\n")
            f.write("=" * 50 + "\n")
            f.write(f"作成日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            if 'actual_free_thickness' in df.columns:
                f.write(f"自由領域厚さ統計:\n")
                f.write(f"  平均: {df['actual_free_thickness'].mean():.1f}Å\n")
                f.write(f"  標準偏差: {df['actual_free_thickness'].std():.1f}Å\n")
                f.write(f"  範囲: {df['actual_free_thickness'].min():.1f} - {df['actual_free_thickness'].max():.1f}Å\n\n")
            
            if 'free_atoms' in df.columns and 'fixed_atoms' in df.columns:
                df['free_ratio'] = df['free_atoms'] / (df['free_atoms'] + df['fixed_atoms'])
                f.write(f"自由原子比率統計:\n")
                f.write(f"  平均: {df['free_ratio'].mean()*100:.1f}%\n")
                f.write(f"  標準偏差: {df['free_ratio'].std()*100:.1f}%\n")
                f.write(f"  範囲: {df['free_ratio'].min()*100:.1f} - {df['free_ratio'].max()*100:.1f}%\n\n")
            
            f.write(f"統一性評価:\n")
            if 'actual_free_thickness' in df.columns and df['actual_free_thickness'].mean() != 0:
                cv_thickness = df['actual_free_thickness'].std() / df['actual_free_thickness'].mean()
                f.write(f"  自由領域厚さの変動係数: {cv_thickness:.3f}\n")
                if cv_thickness < 0.1:
                    f.write(f"  → 非常に良い統一性\n")
                elif cv_thickness < 0.2:
                    f.write(f"  → 良い統一性\n")
                else:
                    f.write(f"  → 改善の余地あり\n")
        
        print(f"比較レポートを作成しました: {report_path}")
    
    return results_data

if __name__ == "__main__":
    main()

NMC界面構造MD解析システム
統合NMC-Al界面構造生成システムとの連携版
発見された界面構造ファイル数: 242
解析対象の界面構造: 242個

処理開始: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.cif
材料: {'al_component': 'Al', 'nmc_component': 'NMC111', 'nmc_variant': 'extreme_Ni_rich_(1, 0, 0)', 'miller_al': '110', 'miller_nmc': '100', 'base_name': 'Al_NMC111_extreme_Ni_rich_(1, 0, 0)'}

--- 条件: P0.001_T300K_HT500K ---
圧着シミュレーション実行中...


The MD trajectory will be saved at /tmp/matlantis_ctlcols3/tmpuqn5o74_.traj.
Note: The max disk size of / is about 30G.
Note: The MD trajectory is saved in a temporary directory, which will be automatically deleted later.


自由領域統一固定層設定:
  総材料厚さ: 47.94Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 31.75Å
  Al領域: 15.00 - 32.80Å
  NMC領域: 30.70 - 62.94Å
  自由領域: 17.00 - 49.75Å
  実際の自由領域厚さ: 32.75Å
  固定原子数: 225/648 (34.7%)
  自由原子数: 423/648 (65.3%)
Dyn step    0 a 12.00 b 12.00 c 77.94


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/md_analysis/compression/Al_NMC111_extreme_Ni_rich_(1, 0, 0)_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


Dyn step 1000 a 12.00 b 12.00 c 77.94
Dyn step 1000 a 12.00 b 12.00 c 77.94
Dyn step 1000 a 12.00 b 12.00 c 77.94
An error occurred processing Al_NMC111_extreme_Ni_rich_(1, 0, 0) (P0.001_T300K_HT500K): ValueError: cannot convert float NaN to integer
=== When reporting an error, please also share the data below: ===
----------------------------------------------------------------------------
time: 2025-08-23T07:20:28.201581+00:00
pid: 15884
code: INVALID_ARGUMENT
method: /pfp.Estimator/Estimate
details: ValueError: cannot convert float NaN to integer
exc: <_InactiveRpcError=20of=20RPC=20that=20terminated=20with:
=09status=20=3D=20StatusCode.INVALID_ARGUMENT
=09details=20=3D=20"ValueError:=20cannot=20convert=20float=20NaN=20to=20int=
eger"
=09debug_error_string=20=3D=20"UNKNOWN:Error=20received=20from=20peer=20=20=
{created_time:"2025-08-23T07:20:28.201135797+00:00",=20grpc_status:3,=20grp=
c_message:"ValueError:=20cannot=20convert=20float=20NaN=20to=20integer"}"
>
notebook_id: 2o2j5wl1z

...

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/md_analysis/compression/Al_NMC111_extreme_Ni_rich_(1, 0, 0)_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


Dyn step 1000 a 12.00 b 12.00 c 77.94
Dyn step 1000 a 12.00 b 12.00 c 77.94
Dyn step 1000 a 12.00 b 12.00 c 77.94
An error occurred processing Al_NMC111_extreme_Ni_rich_(1, 0, 0) (P0.001_T300K_HT800K): Cell is too small
=== When reporting an error, please also share the data below: ===
----------------------------------------------------------------------------
time: 2025-08-23T07:23:51.026027+00:00
pid: 15884
code: INVALID_ARGUMENT
method: /pfp.Estimator/Estimate
details: Cell is too small
exc: <_InactiveRpcError=20of=20RPC=20that=20terminated=20with:
=09status=20=3D=20StatusCode.INVALID_ARGUMENT
=09details=20=3D=20"Cell=20is=20too=20small"
=09debug_error_string=20=3D=20"UNKNOWN:Error=20received=20from=20peer=20=20=
{created_time:"2025-08-23T07:23:51.025671372+00:00",=20grpc_status:3,=20grp=
c_message:"Cell=20is=20too=20small"}"
>
notebook_id: 2o2j5wl1za9jq3qw
metadata: [('client-process-id',=20'15884'),=20('n_atoms',=20'648'),=20('mo=
del_version',=20'v7.0.0'),=20('calc_mode',=20'6')

Traceback (most recent call last):
  File "/tmp/ipykernel_15884/3087055942.py", line 586, in main
    run_interface_compression_md(ase_atoms, pressure, comp_temp, compression_file)
  File "/tmp/ipykernel_15884/3087055942.py", line 783, in run_interface_compression_md
    md(system, extensions=[(PrintCellShape(), 500)])
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/base.py", line 131, in decorated
    res = derived_call(self, *args_list, **kwargs_new)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/md.py", line 353, in __call__
    sim.run(n_run_steps=self.n_run)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/ase_simulation.py", line 192, in run
    dyn.run(steps=n_run_steps)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py", line 315, in run
    for complete in self.irun(steps):
  File "/home

自由領域統一固定層設定:
  総材料厚さ: 47.94Å
  目標自由領域厚さ: 36.00Å
  界面構造検出: 界面位置 = 31.75Å
  Al領域: 15.00 - 32.80Å
  NMC領域: 30.70 - 62.94Å
  自由領域: 17.00 - 49.75Å
  実際の自由領域厚さ: 32.75Å
  固定原子数: 225/648 (34.7%)
  自由原子数: 423/648 (65.3%)
Dyn step    0 a 12.00 b 12.00 c 77.94


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/md_analysis/compression/Al_NMC111_extreme_Ni_rich_(1, 0, 0)_P0.010_T300K_compression.traj.


In [31]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
統合NMC-Al界面構造生成システムで生成された構造に対応

主な修正点:
1. 入力ファイル形式をCIFに変更
2. NMC組成バリアント名に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
"""

from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pathlib
from ase import units
from ase.build import add_vacuum

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# =============================================

# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
INPUT_DIR = os.path.join(BASE_DIR, "structure", "output", "final_interfaces")

# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "md_analysis")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# =============================================
# 統合システム対応のファイルパス関数
# =============================================

def get_interface_files():
    """統合システムで生成された界面構造ファイルを取得"""
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.cif')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出
            # 例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
            if "Interface_" in cif_file and "_on_" in cif_file:
                interface_files.append(cif_file)
    
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """界面構造ファイル名から材料情報を解析"""
    # ファイル名例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
    parts = filename.replace('.xyz', '').split('_')
    
    # デフォルト値
    al_component = "Al"
    nmc_component = "NMC111"
    nmc_variant = "pristine"
    miller_al = "111"
    miller_nmc = "001"
    
    try:
        # "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed" を解析
        if len(parts) >= 6:
            # Al部分の抽出
            al_idx = parts.index("Al")
            if al_idx + 1 < len(parts):
                miller_al = parts[al_idx + 1]
            
            # "on"以降のNMC部分を解析
            on_idx = parts.index("on")
            if on_idx + 1 < len(parts):
                nmc_part = parts[on_idx + 1:]
                
                # NMC基本名
                if "NMC111" in nmc_part[0]:
                    nmc_component = nmc_part[0]
                    
                    # バリアント名の構築
                    if len(nmc_part) > 1:
                        variant_parts = []
                        for i in range(1, len(nmc_part)):
                            if nmc_part[i] not in ["trimmed"] and not nmc_part[i].isdigit():
                                variant_parts.append(nmc_part[i])
                        
                        if variant_parts:
                            nmc_variant = "_".join(variant_parts)
                        
                        # Miller指数の抽出（最後の数値部分）
                        for part in reversed(nmc_part):
                            if part.isdigit() and len(part) == 3:
                                miller_nmc = part
                                break
    
    except (ValueError, IndexError) as e:
        print(f"ファイル名解析警告 ({filename}): {e}")
    
    return {
        'al_component': al_component,
        'nmc_component': nmc_component,
        'nmc_variant': nmc_variant,
        'miller_al': miller_al,
        'miller_nmc': miller_nmc,
        'base_name': f"{al_component}_{nmc_component}_{nmc_variant}"
    }

def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

def get_compression_file_path(base_name, pressure, comp_temp):
    """圧着シミュレーションファイルのパスを取得"""
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

# 界面構造に最適化された統一条件設定
# UNIFIED_CONDITIONS = {
#     "strain_rate": 0.005,             # 界面構造用に低速化（1/ps）
#     "vacuum_top": 15.0,               # 界面構造用vacuum（Å）
#     "temperature": 300.0,             # 引張温度固定（K）
#     "max_strain": 2.0,                # 界面用最大ひずみ
#     "timestep": 1.0,                  # fs
#     "elastic_region": (0.003, 0.02),  # 界面用弾性領域
#     "quality_threshold": 0.85,        # 界面用R²値閾値
#     "max_steps": 50000,               # 界面用最大ステップ数
#     "output_freq": 50,                # 出力頻度
#     "free_region_thickness": 20.0     # 自由領域の統一厚さ（Å）
# }
UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,             # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,               # 界面構造用vacuum（Å）
    "temperature": 300.0,             # 引張温度固定（K）
    "max_strain": 3.0,                # 界面用最大ひずみ
    "timestep": 1.0,                  # fs
    "elastic_region": (0.005, 0.03),  # 界面用弾性領域
    "quality_threshold": 0.85,        # 界面用R²値閾値
    "max_steps": 10000,               # 界面用最大ステップ数
    "output_freq": 50,                # 出力頻度
    "free_region_thickness": 10       # 界面構造用固定層厚さ（Å）
}


# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,            # 界面分離判定の隙間（Å）
    "max_strain_limit": 3.0,          # 最大ひずみ制限
    "min_stress_threshold": -0.5,     # 最小応力閾値（GPa）
    "separation_wait_steps": 300,     # 分離後の待機ステップ
    "stress_drop_ratio": 0.7          # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
compression_temperatures = [300]      # 圧着シミュレーション温度（K）
compression_pressures = [0.001, 0.01] # 圧着圧力（GPa）- サンプリング
high_temperatures = [500, 800]        # 高温シミュレーション温度（K）- サンプリング
tensile_temperature = 300.0           # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とNMC原子（Li, Ni, Mn, Co, O）を分離
    al_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Al']
    nmc_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['Li', 'Ni', 'Mn', 'Co', 'O']]
    
    if not al_indices or not nmc_indices:
        # 界面でない場合のフォールバック
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)),
            'nmc_region': (np.mean(z_coords), np.max(z_coords)),
            'is_interface': False
        }
    
    al_z_coords = z_coords[al_indices]
    nmc_z_coords = z_coords[nmc_indices]
    
    # Al領域とNMC領域の境界を特定
    al_center = np.mean(al_z_coords)
    nmc_center = np.mean(nmc_z_coords)
    
    if al_center < nmc_center:
        # Alが下、NMCが上
        interface_z = (np.max(al_z_coords) + np.min(nmc_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    else:
        # NMCが下、Alが上
        interface_z = (np.max(nmc_z_coords) + np.min(al_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,
        'nmc_region': nmc_region,
        'al_center': al_center,
        'nmc_center': nmc_center,
        'is_interface': True
    }

def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定
    
    Parameters:
    -----------
    atoms : ase.Atoms
        対象の原子構造
    target_free_thickness : float
        自由領域の目標厚さ（Å）
        
    Returns:
    --------
    atoms : ase.Atoms
        固定制約が設定された原子構造
    """
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得
    interface_info = identify_interface_region(atoms)
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  Al領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å")
        print(f"  NMC領域: {interface_info['nmc_region'][0]:.2f} - {interface_info['nmc_region'][1]:.2f}Å")
        
        # 界面中心を基準に自由領域を設定
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
        
        # 自由領域の範囲
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)  # 最低2Å下は固定
        free_z_max = min(free_z_max, z_max - 2.0)  # 最低2Å上は固定
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        # 固定原子の選択（自由領域外を固定）
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合：中心を基準に自由領域を設定
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        # 固定原子の選択
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
    # 固定制約の設定
    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        return atoms
    
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    # 統計情報
    free_atoms = len(atoms) - len(indices)
    free_ratio = free_atoms / len(atoms)
    fixed_ratio = len(indices) / len(atoms)
    
    print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
    print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
    # 構造情報を属性として保存
    atoms.info['free_region_thickness'] = actual_free_thickness
    atoms.info['free_region_bounds'] = (free_z_min, free_z_max)
    atoms.info['fixed_atoms_count'] = len(indices)
    atoms.info['free_atoms_count'] = free_atoms
    
    return atoms

# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

# [既存のComprehensiveTensileAnalyzer等のクラスはそのまま使用]
# [必要に応じて界面構造用にパラメータを調整]

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_nmc_separation = False
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出"""
        
        # Al-NMC界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms)
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region']
                nmc_region = interface_info['nmc_region']
                
                # Al-NMC間の隙間を計算
                if interface_info['al_center'] < interface_info['nmc_center']:
                    gap = nmc_region[0] - al_region[1]
                else:
                    gap = al_region[0] - nmc_region[1]
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_nmc_separation:
                    self.al_nmc_separation = True
                    self.debonding_step = system.current_total_step
                    print(f"      🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        # [界面特有の解析を追加]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] * 160.21766208
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算（界面用に調整されたパラメータ）
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15:  # 界面用に閾値調整
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"      ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_nmc_separation:
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_nmc_separation,
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理"""
    
    print("=" * 80)
    print("NMC界面構造MD解析システム")
    print("統合NMC-Al界面構造生成システムとの連携版")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        print("統合NMC-Al界面構造生成システムを先に実行してください")
        return
    
    print(f"解析対象の界面構造: {len(interface_files)}個")
    
    # 処理対象のサンプリング（全て処理する場合は以下をコメントアウト）
    # interface_files = interface_files[:5]  # 最初の5個のみ処理
    
    processed_count = 0
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"材料: {material_info}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # 既存のMD処理ロジックをここに適用
                            # [圧着 → 平衡化 → 高温 → 引張]の順序で実行
                            
                            compression_file = get_compression_file_path(base_name, pressure, comp_temp)
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                            
                            # ステップ0: 圧着（CIFファイルから読み込み）
                            if not check_md_completed(compression_file, 8000, 100):
                                print(f"圧着シミュレーション実行中...")
                                
                                # CIFファイルの読み込み
                                ase_atoms = read(input_file)
                                
                                # 界面構造に適した固定層設定
                                ase_atoms = set_interface_fixed_atoms(ase_atoms, 
                                    UNIFIED_CONDITIONS["free_region_thickness"])
                                
                                ase_atoms = rotate_atoms_to_lower(ase_atoms)
                                # ase_atoms = set_fixed_atoms_at_bottom(ase_atoms, z_threshold=2.0)
                                indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                                c = FixAtoms(indices)
                                ase_atoms.set_constraint(c)
                                ase_atoms.set_pbc(True)
                                ase_atoms.wrap()
                                atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                                atoms.rotate_atoms_to_upper()
                                t_step = 1.0
                                press_ase = pressure * units.GPa
                                atoms.calc=get_calculator(estimator_fn)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(comp_temp)
                                nvt_integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                        
                                md = MDFeature(
                                    nvt_integrator, 
                                    n_run=1000,  
                                    estimator_fn=estimator_fn
                                )
                                md(system, extensions=[(info, 200)])
                                integrator = NPTIntegrator(
                                    timestep=t_step,
                                    temperature=comp_temp,
                                    pressure=press_ase,
                                    ttime=20*units.fs,
                                    pfactor=2e6*units.GPa*(units.fs**2),
                                    mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
                                )
                                
                                info = PrintCellShape()
                                
                                md = MDFeature(
                                    integrator, 
                                    n_run=8000, 
                                    traj_file_name=compression_file,
                                    traj_freq=100, 
                                    estimator_fn=estimator_fn
                                )
                                
                                md(system, extensions=[(info, 500)])
                                print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) completed for {base_name}")
                            else:
                                print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) already completed for {base_name}")
                            
                            # # 各高温シミュレーション温度に対して処理
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                                
                            try:
                                # ステップ1: 短い平衡化（5 ps）
                                if not check_md_completed(equil_file, 5000, 100):
                                    print(f"Running equilibration for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                                    
                                    if os.path.exists(compression_file):
                                        ase_atoms = read(compression_file, "-1")
                                    else:
                                        ase_atoms = read(input_file)
                                    
                                    ase_atoms = rotate_atoms_to_lower(ase_atoms)
                                    indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                                    c = FixAtoms(indices)
                                    ase_atoms.set_constraint(c)
                                    cell = ase_atoms.get_cell()
                                    
                                    ase_atoms.set_cell(cell, scale_atoms=True)
                                    ase_atoms.wrap()
                                    # type(ase_atoms)
                                    # aroms=ase_atoms
                                    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                                    atoms.rotate_atoms_to_upper()
                                    system = ASEMDSystem(atoms)
                                    system.init_temperature(300)
                                    
                                    integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                    info = PrintCellShape()
                                    
                                    md = MDFeature(
                                        integrator, 
                                        n_run=3000, 
                                        traj_file_name=equil_file,
                                        traj_freq=100, 
                                        estimator_fn=estimator_fn
                                    )
                                    
                                    md(system, extensions=[(info, 1000)])
                                    print(f"Equilibration completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                                else:
                                    print(f"Equilibration already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                                # ステップ2: 高温シミュレーション（可変温度、10ps）
                                if not check_md_completed(high_temp_file, 10000, 100):
                                    print(f"Running high temperature simulation ({high_temp}K) for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                                    
                                    ase_atoms = read(equil_file, "-1")
                                    ase_atoms = rotate_atoms_to_lower(ase_atoms)
                                    indices = [atom.index for atom in ase_atoms if atom.position[2] <= 2.0]
                                    c = FixAtoms(indices)
                                    ase_atoms.set_constraint(c)
                                    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                                    atoms.rotate_atoms_to_upper()
                                    system = ASEMDSystem(atoms)
                                    system.init_temperature(high_temp)
                                    
                                    integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                    info = PrintCellShape()
                                    
                                    md = MDFeature(
                                        integrator, 
                                        n_run=10000, 
                                        traj_file_name=high_temp_file,
                                        traj_freq=100, 
                                        estimator_fn=estimator_fn
                                    )
                                    
                                    md(system, extensions=[(info, 500)])
                                    print(f"High temperature simulation ({high_temp}K) completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                                else:
                                    print(f"High temperature simulation ({high_temp}K) already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        
                        # ステップ3: 早期終了機能付き引張シミュレーション（温度300K固定）
                        analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                        summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                        
                        # 計算済みチェック
                        if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                            print(f"Tensile simulation already completed for {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K)")
                            continue
                        
                        print(f"\n{'='*80}")
                        print(f"早期終了機能付き引張計算開始: {base_name}")
                        print(f"条件: P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K")
                        print(f"統一ひずみ速度: {UNIFIED_CONDITIONS['strain_rate']} 1/ps")
                        print(f"{'='*80}")
                        
                        # 入力ファイル確認
                        if not os.path.exists(high_temp_file):
                            print(f"❌ ステップ2ファイルが見つかりません: {high_temp_file}")
                            continue
                            
                            # ステップ3: 引張（界面用解析器を使用）
                            if not check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                                print(f"引張シミュレーション実行中（界面構造用）...")
                                
                                # 界面構造用の準備と解析器
                                print(f"\n📋 Step 1: 構造準備（vacuum設定: {UNIFIED_CONDITIONS['vacuum_top']}Å）")
                                ase_atoms, initial_reference, structure_info = prepare_structure_with_top_vacuum(
                                    high_temp_file, 
                                    UNIFIED_CONDITIONS["vacuum_top"],
                                    UNIFIED_CONDITIONS["fixed_layer_thickness"]
                                )
                                
                                # 2. MD設定
                                print(f"\n📋 Step 2: MD計算設定（統一条件）")
                                atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                                atoms.rotate_atoms_to_upper()
                                system = ASEMDSystem(atoms)
                                system.init_temperature(UNIFIED_CONDITIONS["temperature"])
                                
                                # 引張設定
                                strain_rate = UNIFIED_CONDITIONS["strain_rate"]
                                max_strain = UNIFIED_CONDITIONS["max_strain"]
                                timestep = UNIFIED_CONDITIONS["timestep"]
                                total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
                                total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
                                
                                if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                                    # vacuum有り：材料長基準の最終長さ
                                    final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
                                    final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
                                else:
                                    # vacuum無し：セル長基準の最終長さ
                                    final_cell_length = initial_reference * (1 + max_strain)
                                
                                latt = np.array(system.ase_atoms.cell)
                                latt[2, 2] = final_cell_length
                                
                                print(f"  統一ひずみ速度: {strain_rate} 1/ps")
                                print(f"  総ステップ数: {total_steps}")
                                print(f"  推定時間: {total_steps * timestep / 1000:.1f} ps")
                                print(f"  ひずみ計算方法: {structure_info['strain_method']}")
                                
                                # メインループ内の修正箇所
        
                                # ステップ3: 解析器セットアップ（早期終了無効化版）
                                print(f"\n📋 Step 3: 解析器セットアップ（早期終了無効）")
                                # analyzer = ComprehensiveTensileAnalyzer(  # WithEarlyStop を削除
                                #     initial_reference, structure_info, UNIFIED_CONDITIONS, high_temp_file, analysis_file
                                # )
                                # 界面用解析器の使用
                                analyzer = InterfaceComprehensiveTensileAnalyzer(
                                    initial_reference, structure_info, UNIFIED_CONDITIONS, 
                                    high_temp_file, analysis_file
                                )
                                 # 4. MD実行（早期終了無効化版）
                                print(f"\n📋 Step 4: MD計算実行（早期終了無効）")
                                integrator = LangevinIntegrator(
                                    timestep=timestep, 
                                    temperature=UNIFIED_CONDITIONS["temperature"]
                                )
                                deform = DeformScheduler(latt, total_steps)
                                info = PrintCellShape()
                                
                                md = MDFeature(
                                    integrator,
                                    n_run=total_steps,
                                    traj_file_name=tensile_file,
                                    traj_freq=UNIFIED_CONDITIONS["output_freq"],
                                    estimator_fn=estimator_fn
                                )
                        
                                extensions = [
                                    (info, 100),
                                    (deform, 1),
                                    (analyzer, 10)
                                ]
                                
                                # 早期終了無効化のMD実行
                                execution_info = run_md_without_early_termination(md, system, extensions)
                                
                                # 5. 結果解析
                                print(f"\n📋 Step 5: 結果解析")
                                final_results = analyzer.get_final_results()
                                final_results['execution_info'] = execution_info
                                
                                # 6. 結果出力（修正版関数を使用）
                                print(f"\n📋 Step 6: 結果出力")
                                save_tensile_results_no_early_termination(  # 新しい関数名
                                    final_results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info
                                )
                                
                                print(f"\n✅ 引張計算完了（早期終了無効）: {base_name}")
                                print(f"📊 結果サマリー:")
                                print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                                print(f"  統一ひずみ速度: {final_results['conditions']['strain_rate']} 1/ps")
                                
                                if final_results['fracture_detected']:
                                    print(f"  破断検出: Step {final_results['fracture_step']} (記録のみ)")
                                if final_results['complete_separation']:
                                    print(f"  完全分離: Step {final_results['separation_step']} (記録のみ)")
                                    
                                print(f"All simulations completed successfully for {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K)")
                            except Exception as e:
                                print(f"An error occurred processing {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K): {e}")
                                import traceback
                                traceback.print_exc()
                                continue

            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def rotate_atoms_to_lower(atoms):
    """原子系のZ座標を確認し、全原子を下端（Z=0付近）に移動させる"""
    ase_atoms = atoms.ase_atoms if hasattr(atoms, 'ase_atoms') else atoms
    
    positions = ase_atoms.get_positions()
    z_min = np.min(positions[:, 2])
    z_max = np.max(positions[:, 2])
    
    shift_z = -z_min
    translation = np.array([0, 0, shift_z])
    ase_atoms.translate(translation)
    
    thickness = z_max - z_min
    new_cell = ase_atoms.get_cell()
    new_cell[2, 2] = z_max + shift_z + 0.5
    ase_atoms.set_cell(new_cell, scale_atoms=False)
    ase_atoms.wrap()
    
    print(f"原子をZ方向に {shift_z:.2f}Å 移動しました")
    print(f"材料厚さ: {thickness:.2f}Å")
    
    return ase_atoms


def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
    """界面構造をMD計算用に準備（自由領域厚み統一版）"""
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # CIFファイルの読み込み
    ase_atoms = read(input_file)
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    
    # 構造解析
    positions = ase_atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max - z_min
    
    print(f"界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f}Å")
    print(f"  原子数: {len(ase_atoms)}")
    
    # 界面情報取得
    interface_info = identify_interface_region(ase_atoms)
    
    if vacuum_top > 0:
        print(f"設定: vacuum有り ({vacuum_top}Å)")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])
        
        new_cell_z = material_thickness + vacuum_top
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()
        
        initial_reference = material_thickness
    else:
        print(f"設定: vacuum無し")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])
        
        margin = 2.0
        new_cell_z = material_thickness + margin
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()
        
        initial_reference = new_cell_z
    
    # 自由領域厚み統一の固定層設定
    ase_atoms = set_interface_fixed_atoms_uniform_free(ase_atoms, target_free_thickness)
    
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_top,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z,
        "interface_info": interface_info,
        "strain_method": "material_length" if vacuum_top > 0 else "cell_length",
        "free_region_thickness": ase_atoms.info.get('free_region_thickness', target_free_thickness),
        "free_region_bounds": ase_atoms.info.get('free_region_bounds', (0, target_free_thickness)),
        "fixed_atoms_count": ase_atoms.info.get('fixed_atoms_count', 0),
        "free_atoms_count": ase_atoms.info.get('free_atoms_count', 0)
    }
    
    print(f"準備完了:")
    print(f"  セルZ長: {new_cell_z:.2f}Å")
    print(f"  材料占有率: {structure_info['material_ratio']*100:.1f}%")
    print(f"  界面検出: {interface_info['is_interface']}")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    
    return ase_atoms, initial_reference, structure_info


def run_interface_compression_md(ase_atoms, pressure, comp_temp, output_file):
    """界面構造用圧着MD"""
    
    ase_atoms.set_pbc(True)
    ase_atoms.wrap()
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    atoms.rotate_atoms_to_upper()
    
    press_ase = pressure * units.GPa
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    atoms.calc = get_calculator(estimator_fn)
    
    system = ASEMDSystem(atoms)
    system.init_temperature(comp_temp)
    
    # 初期NVT平衡化
    nvt_integrator = LangevinIntegrator(timestep=1.0, temperature=comp_temp)
    md = MDFeature(nvt_integrator, n_run=1000, estimator_fn=estimator_fn)
    md(system, extensions=[(PrintCellShape(), 200)])
    
    # NPT圧着
    integrator = NPTIntegrator(
        timestep=1.0,
        temperature=comp_temp,
        pressure=press_ase,
        ttime=20*units.fs,
        pfactor=2e6*units.GPa*(units.fs**2),
        mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
    )
    
    md = MDFeature(
        integrator, 
        n_run=8000, 
        traj_file_name=output_file,
        traj_freq=100, 
        estimator_fn=estimator_fn
    )
    
    md(system, extensions=[(PrintCellShape(), 500)])
    return output_file


def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD（自由領域厚み統一版）"""
    
    # 構造準備
    ase_atoms, initial_reference, structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["free_region_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    atoms.rotate_atoms_to_upper()
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    
    print(f"自由領域統一引張設定:")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    print(f"  自由原子数: {structure_info['free_atoms_count']}")
    print(f"  固定原子数: {structure_info['fixed_atoms_count']}")
    print(f"  引張ひずみ速度: {strain_rate} 1/ps")
    if initial_reference ==0:
        initial_reference =50
    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"NMC-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  Al成分: {material_info['al_component']}\n")
        f.write(f"  NMC成分: {material_info['nmc_component']}\n")
        f.write(f"  NMC組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  Al Miller指数: {material_info['miller_al']}\n")
        f.write(f"  NMC Miller指数: {material_info['miller_nmc']}\n\n")
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {structure_info.get('free_atoms_count', 0)}\n")
        f.write(f"  固定原子数: {structure_info.get('fixed_atoms_count', 0)}\n")
        f.write(f"  自由原子比率: {structure_info.get('free_atoms_count', 0)/(structure_info.get('free_atoms_count', 0)+structure_info.get('fixed_atoms_count', 1))*100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
        
def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"NMC-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  Al成分: {material_info['al_component']}\n")
        f.write(f"  NMC成分: {material_info['nmc_component']}\n")
        f.write(f"  NMC組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  Al Miller指数: {material_info['miller_al']}\n")
        f.write(f"  NMC Miller指数: {material_info['miller_nmc']}\n\n")
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {structure_info.get('free_atoms_count', 0)}\n")
        f.write(f"  固定原子数: {structure_info.get('fixed_atoms_count', 0)}\n")
        f.write(f"  自由原子比率: {structure_info.get('free_atoms_count', 0)/(structure_info.get('free_atoms_count', 0)+structure_info.get('fixed_atoms_count', 1))*100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
        
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        
        f.write(f"\n自由領域統一システムの利点:\n")
        f.write(f"  統一自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']}Å\n")
        f.write(f"  公平な機械特性比較が可能\n")
        f.write(f"  界面中心を基準とした適応的固定層設定\n")
        f.write(f"  材料厚さの違いを考慮した統一評価\n")
        f.write(f"  計算効率と比較性の両立\n")


# 使用例とテスト関数
def test_uniform_free_region():
    """自由領域統一システムのテスト関数"""
    print("自由領域統一システムのテスト開始")
    print("-" * 50)
    
    # テスト用の仮想界面構造を作成
    from ase import Atoms
    
    # Al部分（下部）
    al_positions = []
    for z in np.arange(0, 10, 2):
        for x in np.arange(0, 6, 2):
            for y in np.arange(0, 6, 2):
                al_positions.append([x, y, z])
    
    # NMC部分（上部）
    nmc_positions = []
    elements = ['Li', 'Ni', 'Mn', 'Co', 'O']
    for z in np.arange(12, 25, 2):
        for x in np.arange(0, 6, 2):
            for y in np.arange(0, 6, 2):
                nmc_positions.append([x, y, z])
    
    # 原子構造作成
    positions = al_positions + nmc_positions
    symbols = ['Al'] * len(al_positions) + elements * (len(nmc_positions) // len(elements) + 1)
    symbols = symbols[:len(positions)]
    
    test_atoms = Atoms(symbols=symbols, positions=positions)
    test_atoms.set_cell([8, 8, 30])
    test_atoms.set_pbc(True)
    
    print(f"テスト構造: {len(test_atoms)} atoms, 厚さ {np.max(test_atoms.positions[:, 2]) - np.min(test_atoms.positions[:, 2]):.1f}Å")
    
    # 自由領域統一固定の適用
    target_free = 20.0
    result_atoms = set_interface_fixed_atoms_uniform_free(test_atoms, target_free)
    
    # 結果確認
    print(f"\n結果:")
    print(f"  目標自由領域厚さ: {target_free}Å")
    print(f"  実際の自由領域厚さ: {result_atoms.info.get('free_region_thickness', 'N/A')}Å")
    print(f"  自由/固定比率: {result_atoms.info.get('free_atoms_count', 0)}/{result_atoms.info.get('fixed_atoms_count', 0)}")
    
    return result_atoms


def create_comparison_report(md_results_dir):
    """自由領域統一による結果比較レポートを作成"""
    
    if not os.path.exists(md_results_dir):
        print(f"MD結果ディレクトリが見つかりません: {md_results_dir}")
        return
    
    summary_files = list(Path(md_results_dir).glob("tensile/*_summary.txt"))
    if not summary_files:
        print("解析結果ファイルが見つかりません")
        return
    
    print(f"自由領域統一比較レポート作成中...")
    
    results_data = []
    for summary_file in summary_files:
        try:
            with open(summary_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # 基本情報の抽出
            result = {'filename': summary_file.stem}
            
            # 自由領域情報の抽出
            if '実際の自由領域厚さ:' in content:
                line = [l for l in content.split('\n') if '実際の自由領域厚さ:' in l][0]
                result['actual_free_thickness'] = float(line.split(':')[1].split('Å')[0].strip())
            
            if '自由原子数:' in content:
                line = [l for l in content.split('\n') if '自由原子数:' in l][0]
                result['free_atoms'] = int(line.split(':')[1].strip())
            
            if '固定原子数:' in content:
                line = [l for l in content.split('\n') if '固定原子数:' in l][0]
                result['fixed_atoms'] = int(line.split(':')[1].strip())
            
            # 機械特性の抽出
            if 'ヤング率:' in content:
                line = [l for l in content.split('\n') if 'ヤング率:' in l][0]
                result['young_modulus'] = float(line.split(':')[1].split('GPa')[0].strip())
            
            results_data.append(result)
            
        except Exception as e:
            print(f"ファイル処理エラー {summary_file.name}: {e}")
    
    # 統計レポート
    if results_data:
        df = pd.DataFrame(results_data)
        
        report_path = Path(md_results_dir).parent / "free_region_comparison_report.txt"
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("自由領域統一システム比較レポート\n")
            f.write("=" * 50 + "\n")
            f.write(f"作成日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            if 'actual_free_thickness' in df.columns:
                f.write(f"自由領域厚さ統計:\n")
                f.write(f"  平均: {df['actual_free_thickness'].mean():.1f}Å\n")
                f.write(f"  標準偏差: {df['actual_free_thickness'].std():.1f}Å\n")
                f.write(f"  範囲: {df['actual_free_thickness'].min():.1f} - {df['actual_free_thickness'].max():.1f}Å\n\n")
            
            if 'free_atoms' in df.columns and 'fixed_atoms' in df.columns:
                df['free_ratio'] = df['free_atoms'] / (df['free_atoms'] + df['fixed_atoms'])
                f.write(f"自由原子比率統計:\n")
                f.write(f"  平均: {df['free_ratio'].mean()*100:.1f}%\n")
                f.write(f"  標準偏差: {df['free_ratio'].std()*100:.1f}%\n")
                f.write(f"  範囲: {df['free_ratio'].min()*100:.1f} - {df['free_ratio'].max()*100:.1f}%\n\n")
            
            f.write(f"統一性評価:\n")
            if 'actual_free_thickness' in df.columns:
                cv_thickness = df['actual_free_thickness'].std() / df['actual_free_thickness'].mean()
                f.write(f"  自由領域厚さの変動係数: {cv_thickness:.3f}\n")
                if cv_thickness < 0.1:
                    f.write(f"  → 非常に良い統一性\n")
                elif cv_thickness < 0.2:
                    f.write(f"  → 良い統一性\n")
                else:
                    f.write(f"  → 改善の余地あり\n")
        
        print(f"比較レポートを作成しました: {report_path}")
        f.write(f"NMC-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  Al成分: {material_info['al_component']}\n")
        f.write(f"  NMC成分: {material_info['nmc_component']}\n")
        f.write(f"  NMC組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  Al Miller指数: {material_info['miller_al']}\n")
        f.write(f"  NMC Miller指数: {material_info['miller_nmc']}\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
        
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        return results_data 


if __name__ == "__main__":
    main()

SyntaxError: expected 'except' or 'finally' block (1773243988.py, line 724)

In [15]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
統合NMC-Al界面構造生成システムで生成された構造に対応

主な修正点:
1. 入力ファイル形式をCIFに変更
2. NMC組成バリアント名に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
"""

from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pathlib
from ase import units
from ase.build import add_vacuum

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# =============================================

# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
INPUT_DIR = os.path.join(BASE_DIR, "structure", "output", "final_interfaces")

# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "md_analysis")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# =============================================
# 統合システム対応のファイルパス関数
# =============================================

def get_interface_files():
    """統合システムで生成された界面構造ファイルを取得"""
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.cif')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出
            # 例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
            if "Interface_" in cif_file and "_on_" in cif_file:
                interface_files.append(cif_file)
    
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """界面構造ファイル名から材料情報を解析"""
    # ファイル名例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
    parts = filename.replace('.cif', '').split('_')
    
    # デフォルト値
    al_component = "Al"
    nmc_component = "NMC111"
    nmc_variant = "pristine"
    miller_al = "111"
    miller_nmc = "001"
    
    try:
        # "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed" を解析
        if len(parts) >= 6:
            # Al部分の抽出
            al_idx = parts.index("Al")
            if al_idx + 1 < len(parts):
                miller_al = parts[al_idx + 1]
            
            # "on"以降のNMC部分を解析
            on_idx = parts.index("on")
            if on_idx + 1 < len(parts):
                nmc_part = parts[on_idx + 1:]
                
                # NMC基本名
                if "NMC111" in nmc_part[0]:
                    nmc_component = nmc_part[0]
                    
                    # バリアント名の構築
                    if len(nmc_part) > 1:
                        variant_parts = []
                        for i in range(1, len(nmc_part)):
                            if nmc_part[i] not in ["trimmed"] and not nmc_part[i].isdigit():
                                variant_parts.append(nmc_part[i])
                        
                        if variant_parts:
                            nmc_variant = "_".join(variant_parts)
                        
                        # Miller指数の抽出（最後の数値部分）
                        for part in reversed(nmc_part):
                            if part.isdigit() and len(part) == 3:
                                miller_nmc = part
                                break
    
    except (ValueError, IndexError) as e:
        print(f"ファイル名解析警告 ({filename}): {e}")
    
    return {
        'al_component': al_component,
        'nmc_component': nmc_component,
        'nmc_variant': nmc_variant,
        'miller_al': miller_al,
        'miller_nmc': miller_nmc,
        'base_name': f"{al_component}_{nmc_component}_{nmc_variant}"
    }

def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

def get_compression_file_path(base_name, pressure, comp_temp):
    """圧着シミュレーションファイルのパスを取得"""
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

# 界面構造に最適化された統一条件設定
UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,             # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,               # 界面構造用vacuum（Å）
    "temperature": 300.0,             # 引張温度固定（K）
    "max_strain": 3.0,                # 界面用最大ひずみ
    "timestep": 1.0,                  # fs
    "elastic_region": (0.005, 0.03),  # 界面用弾性領域
    "quality_threshold": 0.85,        # 界面用R²値閾値
    "max_steps": 10000,               # 界面用最大ステップ数
    "output_freq": 50,                # 出力頻度
    # "fixed_layer_thickness": 10       # 界面構造用固定層厚さ（Å）
}

# # 統一条件設定（公平な比較のため）
# UNIFIED_CONDITIONS = {
#     "strain_rate": 0.01,              # 1/ps（統一ひずみ速度）
#     "vacuum_top": 0.0,               # Å（上部vacuum）
#     "temperature": 300.0,             # K（引張温度固定）
#     "max_strain": 3.0,                # 最大ひずみ（材料長基準）
#     "timestep": 1.0,                  # fs
#     "elastic_region": (0.005, 0.03),  # 弾性領域（0.5-3.0%）
#     "quality_threshold": 0.9,         # R²値閾値
#     "max_steps": 100000,              # 最大ステップ数
#     "output_freq": 100,               # 出力頻度
#     "fixed_layer_thickness": 15      # 固定層の相対厚さ
# }

# # 早期終了条件設定
# EARLY_STOP_CONDITIONS = {
#     "separation_gap": 10.0,            # Å 完全分離判定の隙間
#     "max_strain_limit": 5.0,          # 最大ひずみ制限
#     "min_stress_threshold": -1.0,     # 最小応力閾値（圧縮転換）
#     "separation_wait_steps": 500,     # 分離後の待機ステップ
#     "stress_drop_ratio": 0.6          # 破断検出用（記録のみ）
# }

# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,            # 界面分離判定の隙間（Å）
    "max_strain_limit": 3.0,          # 最大ひずみ制限
    "min_stress_threshold": -0.5,     # 最小応力閾値（GPa）
    "separation_wait_steps": 300,     # 分離後の待機ステップ
    "stress_drop_ratio": 0.7          # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
compression_temperatures = [300]      # 圧着シミュレーション温度（K）
compression_pressures = [0.001, 0.01] # 圧着圧力（GPa）- サンプリング
high_temperatures = [500, 800]        # 高温シミュレーション温度（K）- サンプリング
tensile_temperature = 300.0           # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とNMC原子（Li, Ni, Mn, Co, O）を分離
    al_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Al']
    nmc_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['Li', 'Ni', 'Mn', 'Co', 'O']]
    
    if not al_indices or not nmc_indices:
        # 界面でない場合のフォールバック
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)),
            'nmc_region': (np.mean(z_coords), np.max(z_coords)),
            'is_interface': False
        }
    
    al_z_coords = z_coords[al_indices]
    nmc_z_coords = z_coords[nmc_indices]
    
    # Al領域とNMC領域の境界を特定
    al_center = np.mean(al_z_coords)
    nmc_center = np.mean(nmc_z_coords)
    
    if al_center < nmc_center:
        # Alが下、NMCが上
        interface_z = (np.max(al_z_coords) + np.min(nmc_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    else:
        # NMCが下、Alが上
        interface_z = (np.max(nmc_z_coords) + np.min(al_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,
        'nmc_region': nmc_region,
        'al_center': al_center,
        'nmc_center': nmc_center,
        'is_interface': True
    }

def set_interface_fixed_atoms(atoms, fixed_layer_thickness=12.0):
    """界面構造に適した固定原子設定"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    
    # 界面情報を取得
    interface_info = identify_interface_region(atoms)
    
    if interface_info['is_interface']:
        print(f"界面構造検出: 界面位置 = {interface_info['interface_z']:.2f} Å")
        print(f"  Al領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f} Å")
        print(f"  NMC領域: {interface_info['nmc_region'][0]:.2f} - {interface_info['nmc_region'][1]:.2f} Å")
        
        # 各領域の端部を固定
        al_region = interface_info['al_region']
        nmc_region = interface_info['nmc_region']
        
        if interface_info['al_center'] < interface_info['nmc_center']:
            # Al下、NMC上の場合
            # Al領域の下端とNMC領域の上端を固定
            lower_threshold = al_region[0] + fixed_layer_thickness
            upper_threshold = nmc_region[1] - fixed_layer_thickness
        else:
            # NMC下、Al上の場合
            # NMC領域の下端とAl領域の上端を固定
            lower_threshold = nmc_region[0] + fixed_layer_thickness
            upper_threshold = al_region[1] - fixed_layer_thickness
        
        # 固定原子の選択
        indices = [
            i for i, z in enumerate(z_coords)
            if z <= lower_threshold or z >= upper_threshold
        ]
    else:
        # 非界面構造の場合は従来の方法
        print(f"非界面構造: 上下端固定")
        lower_threshold = z_min + fixed_layer_thickness
        upper_threshold = z_max - fixed_layer_thickness
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z <= lower_threshold or z >= upper_threshold
        ]
    
    print(f"固定原子数: {len(indices)}/{len(atoms)} ({len(indices)/len(atoms)*100:.1f}%)")
    
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    return atoms

# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

# [既存のComprehensiveTensileAnalyzer等のクラスはそのまま使用]
# [必要に応じて界面構造用にパラメータを調整]

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_nmc_separation = False
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出"""
        
        # Al-NMC界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms)
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region']
                nmc_region = interface_info['nmc_region']
                
                # Al-NMC間の隙間を計算
                if interface_info['al_center'] < interface_info['nmc_center']:
                    gap = nmc_region[0] - al_region[1]
                else:
                    gap = al_region[0] - nmc_region[1]
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_nmc_separation:
                    self.al_nmc_separation = True
                    self.debonding_step = system.current_total_step
                    print(f"      🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        完全分離チェック
        if z_coords is not None:
            if system.current_total_step % 100 == 0:
                fracture_z,gaps=find_largest_gap_z(system.ase_atoms)
                # interface_z = find_interface_z(system.ase_atoms)
                # upper_z = z_coords[z_coords > interface_z]
                # lower_z = z_coords[z_coords <= interface_z]
                # z_median = np.median(z_coords)
                # upper_z = z_coords[z_coords > z_median]
                # lower_z = z_coords[z_coords <= z_median]
                
                if len(upper_z) > 0 and len(lower_z) > 0:
                    # gap = np.min(upper_z) - np.max(lower_z)
                    if np.max(lower_z) >= 8.0:
                        if gap > EARLY_STOP_CONDITIONS["separation_gap"]:
                            if not self.complete_separation:
                                self.complete_separation = True
                                self.separation_step = system.current_total_step
                                print(f"      🔗 完全分離検出: Step {self.separation_step}, 隙間 {gap:.1f}Å（記録のみ）")
                                
                                self.termination_details.update({
                                    "separation_gap": gap,
                                    "separation_step": self.separation_step
                                })
                            
                            self.separation_wait_count += 1

          # 2. 応力急減チェック（破断検出）- 記録のみ
        if len(self.data) > 100 and self.max_stress > 0:
            recent_stresses = [d[2] for d in self.data[-50:]]
            if len(recent_stresses) > 0:
                recent_max_stress = max(recent_stresses)
                stress_ratio = recent_max_stress / self.max_stress
                
                if stress_ratio < EARLY_STOP_CONDITIONS["stress_drop_ratio"]:
                    if not self.fracture_detected:
                        self.fracture_detected = True
                        self.fracture_step = system.current_total_step
                        print(f"      ⚠️  破断検出: Step {self.fracture_step}, 応力比 {stress_ratio:.3f}（記録のみ）")
                        
                        self.termination_details.update({
                            "fracture_stress_ratio": stress_ratio,
                            "fracture_step": self.fracture_step
                        })
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        # [界面特有の解析を追加]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] * 160.21766208
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算（界面用に調整されたパラメータ）
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15:  # 界面用に閾値調整
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"      ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_nmc_separation:
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_nmc_separation,
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理"""
    
    print("=" * 80)
    print("NMC界面構造MD解析システム")
    print("統合NMC-Al界面構造生成システムとの連携版")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        print("統合NMC-Al界面構造生成システムを先に実行してください")
        return
    
    print(f"解析対象の界面構造: {len(interface_files)}個")
    
    # 処理対象のサンプリング（全て処理する場合は以下をコメントアウト）
    # interface_files = interface_files[:5]  # 最初の5個のみ処理
    
    processed_count = 0
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"材料: {material_info}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # 既存のMD処理ロジックをここに適用
                            # [圧着 → 平衡化 → 高温 → 引張]の順序で実行
                            
                            compression_file = get_compression_file_path(base_name, pressure, comp_temp)
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                            
                            # ステップ0: 圧着（CIFファイルから読み込み）
                            if not check_md_completed(compression_file, 10, 100):
                                print(f"圧着シミュレーション実行中...")
                                
                                # CIFファイルの読み込み
                                ase_atoms = read(input_file)
                                
                                # 界面構造に適した固定層設定
                                ase_atoms = set_interface_fixed_atoms(ase_atoms, 
                                    UNIFIED_CONDITIONS["fixed_layer_thickness"])
                                
                                 ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        # ase_atoms = set_fixed_atoms_at_bottom(ase_atoms, z_threshold=2.0)
                        indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                        c = FixAtoms(indices)
                        ase_atoms.set_constraint(c)
                        ase_atoms.set_pbc(True)
                        ase_atoms.wrap()
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        atoms.rotate_atoms_to_upper()
                        t_step = 1.0
                        press_ase = pressure * units.GPa
                        atoms.calc=get_calculator(estimator_fn)
                        system = ASEMDSystem(atoms)
                        system.init_temperature(comp_temp)
                        nvt_integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                
                        md = MDFeature(
                            nvt_integrator, 
                            n_run=1000,  
                            estimator_fn=estimator_fn
                        )
                        md(system, extensions=[(info, 200)])
                        integrator = NPTIntegrator(
                            timestep=t_step,
                            temperature=comp_temp,
                            pressure=press_ase,
                            ttime=20*units.fs,
                            pfactor=2e6*units.GPa*(units.fs**2),
                            mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
                        )
                        
                        info = PrintCellShape()
                        
                        md = MDFeature(
                            integrator, 
                            n_run=8000, 
                            traj_file_name=compression_file,
                            traj_freq=100, 
                            estimator_fn=estimator_fn
                        )
                        
                        md(system, extensions=[(info, 500)])
                        print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) completed for {base_name}")
                    else:
                        print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) already completed for {base_name}")
                    
                    # # 各高温シミュレーション温度に対して処理
                    equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                    high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                    tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                        
                    try:
                        # ステップ1: 短い平衡化（5 ps）
                        if not check_md_completed(equil_file, 5000, 100):
                            print(f"Running equilibration for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                            
                            if os.path.exists(compression_file):
                                ase_atoms = read(compression_file, "-1")
                            else:
                                ase_atoms = read(input_file)
                            
                            ase_atoms = rotate_atoms_to_lower(ase_atoms)
                            indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                            c = FixAtoms(indices)
                            ase_atoms.set_constraint(c)
                            cell = ase_atoms.get_cell()
                            
                            ase_atoms.set_cell(cell, scale_atoms=True)
                            ase_atoms.wrap()
                            # type(ase_atoms)
                            # aroms=ase_atoms
                            atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                            atoms.rotate_atoms_to_upper()
                            system = ASEMDSystem(atoms)
                            system.init_temperature(300)
                            
                            integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                            info = PrintCellShape()
                            
                            md = MDFeature(
                                integrator, 
                                n_run=3000, 
                                traj_file_name=equil_file,
                                traj_freq=100, 
                                estimator_fn=estimator_fn
                            )
                            
                            md(system, extensions=[(info, 1000)])
                            print(f"Equilibration completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        else:
                            print(f"Equilibration already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        
                        # ステップ2: 高温シミュレーション（可変温度、10ps）
                        if not check_md_completed(high_temp_file, 10000, 100):
                            print(f"Running high temperature simulation ({high_temp}K) for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                            
                            ase_atoms = read(equil_file, "-1")
                            ase_atoms = rotate_atoms_to_lower(ase_atoms)
                            indices = [atom.index for atom in ase_atoms if atom.position[2] <= 2.0]
                            c = FixAtoms(indices)
                            ase_atoms.set_constraint(c)
                            atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                            atoms.rotate_atoms_to_upper()
                            system = ASEMDSystem(atoms)
                            system.init_temperature(high_temp)
                            
                            integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                            info = PrintCellShape()
                            
                            md = MDFeature(
                                integrator, 
                                n_run=10000, 
                                traj_file_name=high_temp_file,
                                traj_freq=100, 
                                estimator_fn=estimator_fn
                            )
                            
                            md(system, extensions=[(info, 500)])
                            print(f"High temperature simulation ({high_temp}K) completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        else:
                            print(f"High temperature simulation ({high_temp}K) already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        
                        # ステップ3: 早期終了機能付き引張シミュレーション（温度300K固定）
                        analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                        summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                        
                        # 計算済みチェック
                        if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                            print(f"Tensile simulation already completed for {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K)")
                            continue
                        
                        print(f"\n{'='*80}")
                        print(f"早期終了機能付き引張計算開始: {base_name}")
                        print(f"条件: P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K")
                        print(f"統一ひずみ速度: {UNIFIED_CONDITIONS['strain_rate']} 1/ps")
                        print(f"{'='*80}")
                        
                        # 入力ファイル確認
                        if not os.path.exists(high_temp_file):
                            print(f"❌ ステップ2ファイルが見つかりません: {high_temp_file}")
                            continue
                            
                            # ステップ1: 平衡化
                            # [既存の平衡化処理ロジック]
                            
                            # ステップ2: 高温処理
                            # [既存の高温処理ロジック]
                            
                            # ステップ3: 引張（界面用解析器を使用）
                            if not check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                                print(f"引張シミュレーション実行中（界面構造用）...")
                                
                                # 界面構造用の準備と解析器
                                # [prepare_structure_with_top_vacuum等を使用]
                                
                                # 界面用解析器の使用
                                analyzer = InterfaceComprehensiveTensileAnalyzer(
                                    initial_reference, structure_info, UNIFIED_CONDITIONS, 
                                    high_temp_file, analysis_file
                                )
                                
                                # [MD実行とresults保存]
                                
                                print(f"引張シミュレーション完了")
                                
                        except Exception as e:
                            print(f"❌ 条件 {condition_name} でエラー: {e}")
                            continue
            
            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def rotate_atoms_to_lower(atoms):
    """原子系のZ座標を確認し、全原子を下端（Z=0付近）に移動させる"""
    ase_atoms = atoms.ase_atoms if hasattr(atoms, 'ase_atoms') else atoms
    
    positions = ase_atoms.get_positions()
    z_min = np.min(positions[:, 2])
    z_max = np.max(positions[:, 2])
    
    shift_z = -z_min
    translation = np.array([0, 0, shift_z])
    ase_atoms.translate(translation)
    
    thickness = z_max - z_min
    new_cell = ase_atoms.get_cell()
    new_cell[2, 2] = z_max + shift_z + 0.5
    ase_atoms.set_cell(new_cell, scale_atoms=False)
    ase_atoms.wrap()
    
    print(f"原子をZ方向に {shift_z:.2f}Å 移動しました")
    print(f"材料厚さ: {thickness:.2f}Å")
    
    return ase_atoms


def prepare_interface_structure_for_md(input_file, vacuum_top, fixed_layer_thickness):
    """界面構造をMD計算用に準備"""
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # CIFファイルの読み込み
    ase_atoms = read(input_file)
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    
    # 構造解析
    positions = ase_atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max - z_min
    
    print(f"界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f}Å")
    print(f"  原子数: {len(ase_atoms)}")
    
    # 界面情報取得
    interface_info = identify_interface_region(ase_atoms)
    
    if vacuum_top > 0:
        print(f"設定: vacuum有り ({vacuum_top}Å)")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])
        
        new_cell_z = material_thickness + vacuum_top
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()
        
        initial_reference = material_thickness
    else:
        print(f"設定: vacuum無し")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])
        
        margin = 2.0
        new_cell_z = material_thickness + margin
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()
        
        initial_reference = new_cell_z
    
    # 界面構造用の固定層設定
    ase_atoms = set_interface_fixed_atoms(ase_atoms, fixed_layer_thickness)
    
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_top,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z,
        "interface_info": interface_info,
        "strain_method": "material_length" if vacuum_top > 0 else "cell_length"
    }
    
    print(f"準備完了:")
    print(f"  セルZ長: {new_cell_z:.2f}Å")
    print(f"  材料占有率: {structure_info['material_ratio']*100:.1f}%")
    print(f"  界面検出: {interface_info['is_interface']}")
    
    return ase_atoms, initial_reference, structure_info


def run_interface_compression_md(ase_atoms, pressure, comp_temp, output_file):
    """界面構造用圧着MD"""
    
    ase_atoms.set_pbc(True)
    ase_atoms.wrap()
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    atoms.rotate_atoms_to_upper()
    
    press_ase = pressure * units.GPa
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    atoms.calc = get_calculator(estimator_fn)
    
    system = ASEMDSystem(atoms)
    system.init_temperature(comp_temp)
    
    # 初期NVT平衡化
    nvt_integrator = LangevinIntegrator(timestep=1.0, temperature=comp_temp)
    md = MDFeature(nvt_integrator, n_run=1000, estimator_fn=estimator_fn)
    md(system, extensions=[(PrintCellShape(), 200)])
    
    # NPT圧着
    integrator = NPTIntegrator(
        timestep=1.0,
        temperature=comp_temp,
        pressure=press_ase,
        ttime=20*units.fs,
        pfactor=2e6*units.GPa*(units.fs**2),
        mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
    )
    
    md = MDFeature(
        integrator, 
        n_run=8000, 
        traj_file_name=output_file,
        traj_freq=100, 
        estimator_fn=estimator_fn
    )
    
    md(system, extensions=[(PrintCellShape(), 500)])
    return output_file


def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD"""
    
    # 構造準備
    ase_atoms, initial_reference, structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["fixed_layer_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    atoms.rotate_atoms_to_upper()
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    if initial_reference ==0:
        initial_reference=50
        
    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存"""
    
    with open(summary_file, 'w') as f:
        f.write(f"NMC-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  Al成分: {material_info['al_component']}\n")
        f.write(f"  NMC成分: {material_info['nmc_component']}\n")
        f.write(f"  NMC組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  Al Miller指数: {material_info['miller_al']}\n")
        f.write(f"  NMC Miller指数: {material_info['miller_nmc']}\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
        
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")


if __name__ == "__main__":
    main()

NMC界面構造MD解析システム
統合NMC-Al界面構造生成システムとの連携版
発見された界面構造ファイル数: 242
解析対象の界面構造: 242個

処理開始: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.cif
材料: {'al_component': 'Al', 'nmc_component': 'NMC111', 'nmc_variant': 'extreme_Ni_rich_(1, 0, 0)', 'miller_al': '110', 'miller_nmc': '100', 'base_name': 'Al_NMC111_extreme_Ni_rich_(1, 0, 0)'}

--- 条件: P0.001_T300K_HT500K ---
圧着シミュレーション実行中...
❌ 条件 P0.001_T300K_HT500K でエラー: 'fixed_layer_thickness'

--- 条件: P0.001_T300K_HT800K ---
圧着シミュレーション実行中...
❌ 条件 P0.001_T300K_HT800K でエラー: 'fixed_layer_thickness'

--- 条件: P0.010_T300K_HT500K ---
圧着シミュレーション実行中...
❌ 条件 P0.010_T300K_HT500K でエラー: 'fixed_layer_thickness'

--- 条件: P0.010_T300K_HT800K ---
圧着シミュレーション実行中...
❌ 条件 P0.010_T300K_HT800K でエラー: 'fixed_layer_thickness'

処理開始: Interface_Al_100_(1, 0, 0)_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.cif
材料: {'al_component': 'Al', 'nmc_component': 'NMC111', 'nmc_variant': 'selective_Co_removal_(1, 0, 0)', 'miller_al': '100', 'miller_nmc':

KeyboardInterrupt: 